In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install transformers
!pip install emoji
!pip install sentencepiece
!pip install schedule

In [3]:
#Headers#
import pandas as pd
import numpy as np

#Preprocessing#
import nltk
from nltk.stem.porter import PorterStemmer
import re
import emoji

#Dataset#
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm, trange
from collections import defaultdict

#Model#
import torch
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig, get_linear_schedule_with_warmup
from keras.preprocessing.sequence import pad_sequences

#Misc#
import schedule

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
df_train1_x=pd.read_csv(r"/content/gdrive/MyDrive/SMMSH_2017_dataset/training_set_1_tweets_csv.csv", sep="\t" )
df_test1_x=pd.read_csv(r"/content/gdrive/MyDrive/SMMSH_2017_dataset/evaluation_set_tweets_csv.csv", sep="\t" )
df_trainY=pd.read_csv(r"/content/gdrive/MyDrive/SMMSH_2017_dataset/training_set_1_ids.txt", sep="\t", header=None )
df_testY=pd.read_csv(r"/content/gdrive/MyDrive/SMMSH_2017_dataset/evaluation_set_ids.txt", sep="\t", header=None)
df_train1=df_train1_x.merge(df_trainY, how = "inner", left_on='id', right_on=0)
df_test1=df_test1_x.merge(df_testY, how = "inner", left_on='id', right_on=0)
df_test1=df_test1.dropna(subset=[2])
df_train1=df_train1.dropna(subset=[2])

In [5]:
df_train1[2].isnull().any()

"@zj_marines that's good girl' I'm giving lily Motrin.. She doesn't like Tylenol .."

**Data Preprocessing**

In [6]:
nltk.download('words')
words = set(nltk.corpus.words.words())
stemmer=PorterStemmer()
def preprocess(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",tweet)
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) 
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI)
    tweet = tweet.replace("#", "").replace("_", " ")
    #tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) if w.lower() in words or not w.isalpha())
    #stems = [stemmer.stem(item) for item in tweet]
    return tweet
df_train1['text'] = df_train1['text'].map(lambda x: preprocess(x))
df_test1['text']=df_test1['text'].map(lambda x:preprocess(x))
#re-labelling
df_train1[2] = df_train1[2].replace([1,2,3],[0,1,2])
df_test1[2] = df_test1[2].replace([1,2,3],[0,1,2])

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
df_train1 = df_train1[0:1000]

**Dataset and Dataloader**

In [8]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased', truncation=True)

In [9]:
class DrugIntakeDataset(Dataset):
  def __init__(self, text, targets, tokenizer, max_len):
    self.text = text
    self.targets = targets
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.text)

  def __getitem__(self, item):
    text = str(self.text[item])
    target = self.targets[item]

    encoding = self.tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=self.max_len,
    return_token_type_ids=False,
    pad_to_max_length=False,
    return_attention_mask=True,
    return_tensors='pt',
    )

    input_ids = pad_sequences(encoding['input_ids'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
    input_ids = input_ids.astype(dtype = 'int64')
    input_ids = torch.tensor(input_ids) 

    attention_mask = pad_sequences(encoding['attention_mask'], maxlen=MAX_LEN, dtype=torch.Tensor ,truncating="post",padding="post")
    attention_mask = attention_mask.astype(dtype = 'int64')
    attention_mask = torch.tensor(attention_mask)

    return {
        'text': text,
        'input_ids': input_ids.flatten(),
        'attention_mask': attention_mask.flatten(),
        'targets': torch.tensor(target, dtype=torch.long)
        }

In [ ]:
  # ds = DrugIntakeDataset(
  #   text=df['text'].to_numpy(),
  #   targets=df[2].to_numpy(),
  #   tokenizer=tokenizer,
  #   max_len=max_len
  # )
#  df_train1['text'].to_numpy()

In [11]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = DrugIntakeDataset(
    text=df['text'].to_numpy(),
    targets=df[2].to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
  )

  return DataLoader(
    ds,
    batch_size=batch_size,
    num_workers=2
  )

In [12]:
MAX_LEN=128
BATCH_SIZE = 8
train_data_loader = create_data_loader(df_train1, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test1, tokenizer, MAX_LEN, BATCH_SIZE)

**Model Building**

In [26]:
class XLNet_Classification(torch.nn.Module):
  
  def __init__(self, num_labels=3):
    super(XLNet_Classification, self).__init__()
    self.num_labels = num_labels
    self.xlnet = XLNetModel.from_pretrained('xlnet-base-cased')
    for param in self.xlnet.parameters():
      param.requires_grad = False
    self.classifier = torch.nn.Linear(768, num_labels)

    torch.nn.init.xavier_normal_(self.classifier.weight)

  def forward(self, input_ids, token_type_ids=None,
              attention_mask=None, labels=None):
    # last hidden layer
    last_hidden_state = self.xlnet(input_ids=input_ids,
                                   attention_mask=attention_mask,
                                   token_type_ids=token_type_ids)
    # pool the outputs into a mean vector
    mean_last_hidden_state = self.pool_hidden_state(last_hidden_state)
    logits = self.classifier(mean_last_hidden_state)
        
    if labels is not None:
      loss_fct = CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels),
                      labels.view(-1, self.num_labels))
      return loss
    else:
      return logits
    
  def freeze_xlnet_decoder(self):
    """
    Freeze XLNet weight parameters. They will not be updated during training.
    """
    for param in self.xlnet.parameters():
      param.requires_grad = False
    
  def unfreeze_xlnet_decoder(self):
    """
    Unfreeze XLNet weight parameters. They will be updated during training.
    """
    for param in self.xlnet.parameters():
      param.requires_grad = True
    
  def pool_hidden_state(self, last_hidden_state):
    """
    Pool the output vectors into a single mean vector 
    """
    last_hidden_state = last_hidden_state[0]
    mean_last_hidden_state = torch.mean(last_hidden_state, 1)
    return mean_last_hidden_state

# len(Y_train[0]) = 6


In [29]:
EPOCHS = 10
model = XLNet_Classification(num_labels=3)
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01, correct_bias=False)
loss_fn = CrossEntropyLoss().to(device)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)


**Model Training**

In [30]:
def train_epoch(
  model,
  data_loader,
  loss_fn,
  optimizer,
  device,
  scheduler,
  n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0
    for iter,d in enumerate(data_loader):
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
        model = model.to(device)
        outputs = model(
          input_ids=input_ids,
          attention_mask=attention_mask
        )
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)
        correct_predictions += torch.sum(preds == targets)
        losses.append(loss.item())
        loss.backward()
        clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        print('\r Iter\t: '+str(iter)+'\tLoss\t'+str(loss.item()),end='')
    return correct_predictions.double() / n_examples, np.mean(losses)

In [31]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            model = model.to(device)
            outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)
            correct_predictions += torch.sum(preds == targets)
            losses.append(loss.item())
    return correct_predictions.double() / n_examples, np.mean(losses)

**Training XLNet Model**

In [ ]:
%%time

CUDA_LAUNCH_BLOCKING=1

EPOCHS=10

history = defaultdict(list)
best_accuracy = 0
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)
    train_acc, train_loss = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(df_train1))
    print(f'Train loss {train_loss} accuracy {train_acc}')
    val_acc, val_loss = eval_model(
    model,
    test_data_loader,
    loss_fn,
    device,
    len(df_test1))
    print(f'Val   loss {val_loss} accuracy {val_acc}')
    
    history['train_acc'].append(train_acc)
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_acc)
    history['val_loss'].append(val_loss)
    if val_acc > best_accuracy:
        torch.save(model.state_dict(), 'best_model_state.bin')
        best_accuracy = val_acc

Epoch 1/10
----------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


 Iter	: 124	Loss	5.113805770874023Train loss 5.066668026447296 accuracy 0.249


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Val   loss 3.362833816974416 accuracy 0.3013796810607418
Epoch 2/10
----------


...

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Val   loss 2.3207856622073577 accuracy 0.3666009675685361
Epoch 7/10
----------


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


 Iter	: 124	Loss	2.121192455291748Train loss 1.791858696937561 accuracy 0.419


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
